In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd
import geopandas as gpd
from pandas import DataFrame
from shapely.geometry import Point, Polygon
from shapely.wkt import loads

In [18]:
#100mメッシュの重心
gdf_center_path = '/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市/奥州市　重心/gdf_center.shp'
#土壌
gdf_soil_path = '/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市/奥州市　土壌/土壌ph/土壌ph.csv'
#地形
gdf_geo_path = '/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/地形・表層地質データ/岩手/030地形区分（ポリゴン）.shp'
#表層
gdf_surf_path = '/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/地形・表層地質データ/岩手/030表層地質（ポリゴン）.shp'

In [19]:
#100mメッシュの重心
gdf_center = gpd.read_file(gdf_center_path)

In [20]:
#土壌
df_soil = pd.read_csv(gdf_soil_path)

In [21]:
#地形
gdf_geo = gpd.read_file(gdf_geo_path)

In [22]:
#表層
gdf_surf = gpd.read_file(gdf_surf_path)

In [23]:
#土壌データをgdfに変換

#文字列となったgeometryを変換
df_soil['geometry'] = loads(df_soil['geometry'])

#gdfに変換
gdf_soil = gpd.GeoDataFrame(df_soil, geometry='geometry')

In [24]:
#座標変換
gdf_list = [gdf_center, gdf_soil, gdf_geo, gdf_surf]

for gdf in gdf_list:
  if gdf.crs is None:
  	gdf.set_crs(epsg=4326, inplace=True)

In [25]:
#カラム名を指定の名前に変更
gdf_geo.rename(columns={'属性1':'地形タイプ1', '属性2':'地形タイプ2'}, inplace=True)
gdf_surf.rename(columns={'属性1':'表層タイプ1', '属性2':'表層タイプ2'}, inplace=True)

In [26]:
gdf_soil.drop('index_right', axis=1, inplace=True)

In [27]:
#結合
i = 0
merge_list = [gdf_soil, gdf_geo, gdf_surf]

for gdf in merge_list:
  if i == 0:
    gdf_merge = gpd.sjoin(gdf_center, gdf, op='intersects',  how='left')

  else:
    gdf_merge = gpd.sjoin(gdf_merge, gdf, op='intersects',  how='left')

  #重複を削除
  gdf_merge = gdf_merge[~gdf_merge.index.duplicated()]

  #'index_right'があると結合できないから削除
  gdf_merge.drop('index_right', axis=1, inplace=True)

  i += 1

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-27-6f97582c437a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_merge.drop('index_right', axis=1, inplace=True)
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be re

In [29]:
gdf_merge = gdf_merge[['Meshcode', 'soil_ph', '土壌タイプ1', '土壌タイプ2', '地形タイプ1', '地形タイプ2', '表層タイプ1', '表層タイプ2']]

In [30]:
#出力path
gdf_merge_path = '/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市/奥州市　土壌/土壌タイプ/土壌タイプ.csv'

In [36]:
gdf_merge.to_csv(gdf_merge_path, encoding='cp932', index=False)

In [34]:
gdf_merge

,Meshcode,soil_ph,土壌タイプ1,土壌タイプ2,地形タイプ1,地形タイプ2,表層タイプ1,表層タイプ2
0,5840378485,NaN,褐色森林土,乾性褐色森林土壌,丘陵地,大起伏丘陵地,火山性,流紋岩質岩石（石英安山岩質岩石を含む）
1,5840378486,NaN,褐色森林土,乾性褐色森林土壌,丘陵地,大起伏丘陵地,火山性,流紋岩質岩石（石英安山岩質岩石を含む）
2,5840378487,NaN,褐色森林土,乾性褐色森林土壌,丘陵地,大起伏丘陵地,火山性,流紋岩質岩石（石英安山岩質岩石を含む）
3,5840378495,NaN,褐色森林土,乾性褐色森林土壌,丘陵地,大起伏丘陵地,火山性,流紋岩質岩石（石英安山岩質岩石を含む）
4,5840378496,NaN,褐色森林土,乾性褐色森林土壌,丘陵地,大起伏丘陵地,火山性,流紋岩質岩石（石英安山岩質岩石を含む）
...,...,...,...,...,...,...,...,...
101381,5841728323,NaN,褐色森林土,褐色森林土壌,山地,小起伏山地,深成岩,花崗岩質岩石
101382,5841728324,NaN,褐色森林土,褐色森林土壌,山地,小起伏山地,深成岩,花崗岩質岩石
101383,5841730000,NaN,黒ボク土,黒ボク土壌,山地,中起伏山地,深成岩,蛇紋岩質岩石
101384,5841730001,NaN,黒ボク土,黒ボク土壌,山地,中起伏山地,深成岩,蛇紋岩質岩石
